# how to use bayesian optimization

In [1]:
import pandas as pd
import numpy as np
from src.data.make_dataset import thesis_lookup_objective, thesis_search_space
from src.hyperparameter_analyses.bayesian_optimization import BayesianOptimization
from src.hyperparameter_analyses.bayesian_optimization import Config
from src.visualization.visualize import visualize_search_performance, compare_search_performance

In [2]:
objective = thesis_lookup_objective('WEST_diff_17520')
search_space = thesis_search_space()

In [3]:
bay_opt = BayesianOptimization(
    search_space=search_space,
    objective=objective,
    max_evals=100,
    algo='tpe'
)

In [4]:
results = bay_opt.run_analysis()

100%|██████████| 100/100 [00:09<00:00, 11.06it/s, best loss: 13.550832541979823]
